# Data selection and preprocessing:

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
ratings = pd.read_csv('ratings.csv')


In [3]:
def train_test_split_user(ratings, test_size=0.2):
    train_list = []
    test_list = []

    # Get all unique users
    for user_id in ratings['userId'].unique():
        # Filter the dataset to include only rows corresponding to curr user
        user_data = ratings[ratings['userId'] == user_id]
        # Check if user has rated more than 5 movies to meaningfully split data into train and test
        # Ex. user with 10 ratings -> 8 train, 2 test, but user with 3 ratings -> 2 train, 1 test
            # In the latter case, we would not have enough data to train the model
        # If user has rated more than 5 movies, split the data into train and test
        # Else, include all data in train
        if len(user_data) >= 5:
            train_data, test_data = train_test_split(user_data, test_size=test_size, random_state=42)
            train_list.append(train_data)
            test_list.append(test_data)
        else:
            train_list.append(user_data)
    
    # Combine training and testing data for all users into train and test
    train = pd.concat(train_list)
    test = pd.concat(test_list)
    return train, test

train_data, test_data = train_test_split_user(ratings)
print(train_data.head())
# We don't need timestamp column
train_data = train_data.drop(columns=['timestamp'])
test_data = test_data.drop(columns=['timestamp'])

     userId  movieId  rating  timestamp
55        1     1031     5.0  964982653
230       1     4006     4.0  964982903
69        1     1197     5.0  964981872
168       1     2596     5.0  964981144
109       1     1777     4.0  964981230


In [14]:
train_data['weight'] = train_data.groupby('movieId')['movieId'].transform('count')
train_data['weight'] = train_data['weight'] / train_data['weight'].sum()  # Normalize weights
train_data['rating'] = (train_data['rating'] - train_data['rating'].min()) / \
                       (train_data['rating'].max() - train_data['rating'].min())

sampled_train_data = train_data.sample(frac=1, weights=train_data['weight'], random_state=42)

In [5]:
class MF():

    def __init__(self, R, K, alpha, beta, iterations):
        self.R = R
        self.num_users, self.num_movies = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_movies, self.K))

        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_movies)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_movies)
            if self.R[i, j] > 0
        ]

        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            print("Iteration: %d ; mae = %.4f ; rmse = %.4f" % (i+1, self.mae(), self.rmse()))
          
    def mae(self):
        xs, ys = self.R.nonzero() 
        predicted = self.full_matrix()  
        error = 0
        
        for x, y in zip(xs, ys):
            error += abs(self.R[x, y] - predicted[x, y])  
        
        return error / len(xs)
    
    def rmse(self):
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)

        return np.sqrt(error/len(xs))

    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            P_i = self.P[i, :].copy()
            Q_j = self.Q[j, :].copy()

            self.P[i, :] += self.alpha * (e * Q_j - self.beta * P_i)
            self.Q[j, :] += self.alpha * (e * P_i - self.beta * Q_j)

    def get_rating(self, i, j):
        m_b = self.b_i[j] if j < len(self.b_i) else 0
        l_i = self.Q[j, :].T if j < len(self.Q) else np.mean(self.Q, axis=0) 
        prediction = self.b + self.b_u[i] + m_b + self.P[i, :].dot(l_i)
        return prediction

    def full_matrix(self):
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)


In [6]:
user_ids = sampled_train_data['userId'].unique()  
movie_ids = sampled_train_data['movieId'].unique()  

user_to_idx = {user_id: idx for idx, user_id in enumerate(user_ids)}
movie_to_idx = {movie_id: idx for idx, movie_id in enumerate(movie_ids)}

R = np.zeros((len(user_ids), len(movie_ids)))

for row in sampled_train_data.itertuples():
    user_idx = user_to_idx[row.userId]
    movie_idx = movie_to_idx[row.movieId]
    R[user_idx, movie_idx] = row.rating

mf = MF(R, K=50, alpha=0.01, beta=0.05, iterations=35)
# Iteration: 178 ; mae = 0.1979 ; rmse = 0.2732

mf.train()

Iteration: 1 ; mae = 0.1593 ; rmse = 0.1982
Iteration: 2 ; mae = 0.1548 ; rmse = 0.1933
Iteration: 3 ; mae = 0.1521 ; rmse = 0.1906
Iteration: 4 ; mae = 0.1503 ; rmse = 0.1888
Iteration: 5 ; mae = 0.1489 ; rmse = 0.1872
Iteration: 6 ; mae = 0.1479 ; rmse = 0.1861
Iteration: 7 ; mae = 0.1471 ; rmse = 0.1852
Iteration: 8 ; mae = 0.1464 ; rmse = 0.1844
Iteration: 9 ; mae = 0.1458 ; rmse = 0.1838
Iteration: 10 ; mae = 0.1453 ; rmse = 0.1832
Iteration: 11 ; mae = 0.1448 ; rmse = 0.1826
Iteration: 12 ; mae = 0.1444 ; rmse = 0.1822
Iteration: 13 ; mae = 0.1441 ; rmse = 0.1818
Iteration: 14 ; mae = 0.1437 ; rmse = 0.1814
Iteration: 15 ; mae = 0.1433 ; rmse = 0.1810
Iteration: 16 ; mae = 0.1431 ; rmse = 0.1808
Iteration: 17 ; mae = 0.1427 ; rmse = 0.1804
Iteration: 18 ; mae = 0.1425 ; rmse = 0.1801
Iteration: 19 ; mae = 0.1423 ; rmse = 0.1799
Iteration: 20 ; mae = 0.1420 ; rmse = 0.1796
Iteration: 21 ; mae = 0.1419 ; rmse = 0.1794
Iteration: 22 ; mae = 0.1417 ; rmse = 0.1792
Iteration: 23 ; mae

In [ ]:
def test_model(mf_model, test_data, user_to_idx, movie_to_idx):
    squared_error = 0
    absolute_error = 0
    n = len(test_data)

    for _, row in test_data.iterrows():
        uid = int(row['userId'])
        mid = int(row['movieId'])
             
        user_id = int(user_to_idx[uid])
        movie_id = int(movie_to_idx[mid]) if mid in movie_to_idx else len(movie_to_idx)
        actual_rating = row['rating']
        
        predicted_rating = mf_model.get_rating(user_id, movie_id)
        
        squared_error += (actual_rating - predicted_rating) ** 2
        
        absolute_error += abs(actual_rating - predicted_rating)
    
    rmse = np.sqrt(squared_error / n)
    mae = absolute_error / n

    return rmse, mae

In [11]:
rmse, mae = test_model(mf, test_data, user_to_idx, movie_to_idx)
print("Test RMSE:", rmse)
print("Test MAE:", mae)

Test RMSE: 0.26161755323646674
Test MAE: 0.19770313101697526


In [12]:
def unwatched_movies(ratings):
    all_movie_ids = set(ratings['movieId'])
    user_to_unwatched_movies = {}

    for user_id in ratings['userId'].unique():
        user = ratings[ratings['userId'] == user_id]
        watched_movies = set(user['movieId'])
        unwatched_movies = all_movie_ids - watched_movies
        user_to_unwatched_movies[user_id] = unwatched_movies

    return user_to_unwatched_movies

def recommend_movies(mf_model, user_to_idx, movie_to_idx, n=10):
    unwatched = unwatched_movies(train_data)
    recommendations = {}

    for user_id in user_to_idx.keys():
        user_idx = user_to_idx[user_id]
        predictions = mf_model.full_matrix()[user_idx]
        recommended_movie_idxs = np.argsort(predictions)[::-1]
        recommended_movies = [k for k, v in movie_to_idx.items() if v in recommended_movie_idxs and k in unwatched[user_id]][:n]
        recommendations[user_id] = recommended_movies

    return recommendations

top_10_recommendations = recommend_movies(mf, user_to_idx, movie_to_idx, n=10)

In [13]:
def dcg(recommended_movies, actual_movies):
    dcg_value = 0.0
    for i, movie in enumerate(recommended_movies):
        if movie in actual_movies:
            dcg_value += 1 / np.log2(i + 2)  
    return dcg_value

def idcg(actual_movies, length):
    idcg_value = 0.0
    for i in range(min(len(actual_movies), length)):
        idcg_value += 1 / np.log2(i + 2)
    return idcg_value

def recommendation_performance(recommendations, test_data):
    running_precision, running_recall, running_ndcg = 0, 0, 0

    for user_id, recommended_movies in recommendations.items():
        actual_movies = test_data[test_data['userId'] == user_id]['movieId']

        intersection = len(set(recommended_movies) & set(actual_movies))
        precision = (intersection / len(recommended_movies)) * 100
        recall = (intersection / len(actual_movies)) * 100

        running_precision += precision
        running_recall += recall

        dcg_value = dcg(recommended_movies, actual_movies)
        idcg_value = idcg(actual_movies, len(recommended_movies))
        ndcg = (dcg_value / idcg_value) * 100 if idcg_value > 0 else 0
        running_ndcg += ndcg

    precision = running_precision / len(recommendations)
    recall = running_recall / len(recommendations)
    f_measure = 2 * (precision * recall) / (precision + recall)
    ndcg = running_ndcg / len(recommendations)

    return precision, recall, f_measure, ndcg

print("Precision \t%.4f%%\nRecall \t\t%.4f%%\nF-Measure \t%.4f%%\nNDCG \t\t%.4f" % recommendation_performance(top_10_recommendations, test_data))

Precision 	6.0492%
Recall 		2.2724%
F-Measure 	3.3038%
NDCG 		0.0000
